In [1]:
import re
import requests
from bs4 import BeautifulSoup
import unidecode
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
from slugify import slugify
from opencage.geocoder import OpenCageGeocode
import pymongo
from pymongo import MongoClient
from tqdm import tqdm
from tqdm import trange
import folium
from folium import plugins
from geopy import distance

In [2]:
#creating the Mongo conection
cliente=MongoClient('mongodb://localhost:27017')
db = cliente.gymy
col = db.gymy

In [3]:
def token(token):
    with open(token, 'r') as f:
        t=f.readlines()[0].split('\n')[0]
    return t

In [4]:
#init geocoder
geocoder = OpenCageGeocode(token('geocagetoken.txt'))

In [5]:
#find near geoquery
def find_near(geopoint, radio=1000):
    
    return col.find({'principal':{
        '$near': {'$geometry': geopoint, '$maxDistance':radio}
    }})

In [60]:
#Function that gets the lat long from user input
def getting_coor():
    try:
        user_input = input('¿Donde te gustaría ejercitarte? ')
        meters = 2000
        results = geocoder.geocode(user_input)
        fcoordinates=[]
        coordinates=[]
        coordinates.append(results[0]['geometry']['lng'])
        coordinates.append(results[0]['geometry']['lat']) 
        bandera = results[0]['annotations']['flag']
        country = results[0]['components']['country']
        city=results[0]['components']['city']
        localidad = country+', '+city
        fcoordinates.append(coordinates)
        fcoordinates.append(meters)
        fcoordinates.append(localidad)
        fcoordinates.append(user_input)
        fcoordinates.append(bandera)
        return fcoordinates
    except:
        return 'Not a valid address'

In [154]:
#returns map based on user input with gyms nearby

try:
    coordenadas = getting_coor() #acá inicializa la funcion creada arriba
    longlat = coordenadas[0]
    latlong = longlat[::-1]
    radio = coordenadas[1]
    pais = coordenadas[4]
    localidad = coordenadas[2]
    direccion_solicitud = coordenadas[3]

    address={'type': 'Point', 'coordinates':longlat}  # geopoint

    ratio=radio  # radio de busqueda

    n_gyms=find_near(address, ratio).count()
    gyms = find_near(address, ratio)
    gyms=pd.DataFrame(gyms)

    #creating map object
    tooltip = 'Location you chose: {} \n {}'.format(direccion_solicitud,localidad)
    mapa=folium.Map(latlong, zoom_start=14, width='100%', height='100%')
    datamapa=gyms[['lat', 'long']].values
    folium.Marker(latlong, tooltip=tooltip, icon=folium.Icon()).add_to(mapa)
    for i in trange(len(gyms)):
        popup =''

        if str(gyms.phone[i]) != 'nan' :
            popup += 'Telefono: ' + str(gyms.phone[i])

        if str(gyms.address[i]) != 'nan' :
            popup += ' Direccion: '+str(gyms.address[i])

        if str(gyms.web[i]) != 'nan' :
            popup += ' Web: '+str(gyms.web[i])

        folium.Marker([gyms.lat[i],gyms.long[i]], popup=popup,
                      tooltip = gyms.names[i], icon = folium.Icon(color='red')).add_to(mapa)   
    mapa.save('MapaFinal.html')
    print('Amount of gyms near {} are: {}'.format(direccion_solicitud,n_gyms))
    display(mapa)
    display(gyms)
    
except:
    print('Dirección inválida, por favor ingresa otra dirección')

¿Donde te gustaría ejercitarte? zona 14 guatemala


<ipython-input-154-095a2a2b86e8>:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  n_gyms=find_near(address, ratio).count()
100%|██████████| 30/30 [00:00<00:00, 1014.45it/s]


Amount of gyms near zona 14 guatemala are: 30


,_id,names,address,phone,web,description,category,city,link,long,lat,principal
0,5ebdbf30703b9cbb5ec9b0cd,Vitam CrossFit,5a Avenida 12-56 Zona 14,tel:+(502) 2366-6810,https://directorio.guatemala.com/listing_repor...,El mejor y más completo método de acondicionam...,Crossfit,Guatemala - Guatemala,https://www.google.com.ar/maps/place/5+Avenida...,-90.519826,14.584827,"{'type': 'Point', 'coordinates': [-90.51982609..."
1,5ebdbf30703b9cbb5ec9b0bf,Squash Zone,10 Avenida 10-50 Zona 14,(+502) 5580 1015,NaN,NaN,entrenamiento fisico,Guatemala - Guatemala,https://www.google.com.ar/maps/place/Futeca+Sp...,-90.523611,14.582833,"{'type': 'Point', 'coordinates': [-90.5236105,..."
2,5ebdbf30703b9cbb5ec9b0be,Sporta Gym,La Noria,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/place/Sporta+La+No...,-90.524555,14.581316,"{'type': 'Point', 'coordinates': [-90.52455479..."
3,5ebdbf30703b9cbb5ec9b03e,Club La Villa,20 Avenida 8-81,(+502) 2333 7324,NaN,NaN,deportes,Guatemala - Guatemala,https://www.google.com/maps/place/Club+la+Vill...,-90.512070,14.580398,"{'type': 'Point', 'coordinates': [-90.51206979..."
4,5ebdbf30703b9cbb5ec9b08c,Micheo Boxing by Futeca Zona 14,10a Avenida 10-50 Zona 14,tel:+(502) 2366-8052,NaN,En Micheo Boxing by Futeca somos una academia ...,Boxeo,Guatemala - Guatemala,"https://www.google.com/maps/place/Futeca,+Cdad...",-90.523159,14.584182,"{'type': 'Point', 'coordinates': [-90.5231594,..."
5,5ebdbf30703b9cbb5ec9b07b,Futeca Playball,Km20.8 Carr A El Salvador Fraijanes,(+502) 2424 3939,NaN,NaN,deportes,Guatemala - Guatemala,https://www.google.com/maps/search/futeca+play...,-90.523159,14.584182,"{'type': 'Point', 'coordinates': [-90.5231594,..."
6,5ebdbf30703b9cbb5ec9b088,Invictus Performance Training,"8a. avenida 6-63, Zona 14",2367 1239,https://www.facebook.com/invictusperformance/,crossfit,crossfit,Guatemala - Guatemala,https://www.google.com/maps/place/INVICTUS+Per...,-90.522058,14.585932,"{'type': 'Point', 'coordinates': [-90.5220585,..."
7,5ebdbf30703b9cbb5ec9b092,Okinawan Goju Ryu,Avenida Las Américas 17-16,tel:+(502) 2334-5697,https://directorio.guatemala.com/listing_repor...,Okinawan Goju Ryu en Guatemala es dojo que des...,Artes Marciales,Guatemala - Guatemala,https://www.google.com/maps/search/Avenida+Las...,-90.525786,14.577359,"{'type': 'Point', 'coordinates': [-90.5257858,..."
8,5ebdbf30703b9cbb5ec9b03f,Club Municipal,2 Calle 15-95 Zona 13,(+502) 2333 0037,NaN,NaN,deportes,Guatemala - Guatemala,"https://www.google.com/maps/place/2+Calle+15,+...",-90.514859,14.587253,"{'type': 'Point', 'coordinates': [-90.5148594,..."
9,5ebdbf30703b9cbb5ec9b08e,Monkey Business Calisthenics,"15 Avenida y 4a. Calle 3-15, Zona 14",tel:+(502) 5917-0558,NaN,En Monkey Business Calisthenics verás forma se...,"Calisthenia, TRX",Guatemala - Guatemala,https://www.google.com/maps/place/Monkey+Busin...,-90.522905,14.587550,"{'type': 'Point', 'coordinates': [-90.5229046,..."


In [156]:
gyms.phone[i]

nan

# Trying GEOPY

In [67]:
futeca = (14.582708,-90.5178937)
oakland = (14.5943548,-90.5092745)
print(distance.distance(futeca, oakland).km)

1.5884617079627497


In [71]:
gymy = pd.read_csv('gymy_latlong.csv')
gymy.drop(columns='Unnamed: 0', inplace=True)
gymy.head()

,names,address,phone,web,description,category,city,link,long,lat,principal
0,180 Grados Fitness,Km. 22.5 Carretera a El Salvador,tel:+(502) 6662-5087,https://directorio.guatemala.com/listing_repor...,180 Grados Fitness tiene como objetivo fundame...,"Crossfit, TRX",Guatemala - Guatemala,https://www.google.com.ar/maps/place/180+grado...,-90.488614,14.501430,"{'type': 'Point', 'coordinates': ['-90.4886139..."
1,Academia BellyDance Rashida,"Plaza Las Cañas, Local 201, zona 10, Ciudad de...",2337-0980,https://es-la.facebook.com/Rashidaacademy/,Esta academia de baile es ideal para las perso...,Baile,Guatemala - Guatemala,https://www.google.com.ar/maps/place/Academia+...,-90.505299,14.581305,"{'type': 'Point', 'coordinates': ['-90.5052988..."
2,Academia Dance Art,"Bulevar Vista Hermosa 25-80, zona 15 Vista Her...",3093-8434,https://es-la.facebook.com/Danceartguatemala/,La Academia Dance Art ofrece profesionales exp...,Baile,Guatemala - Guatemala,https://www.google.com.ar/maps/place/edificio+...,-90.493654,14.580998,"{'type': 'Point', 'coordinates': ['-90.4936543..."
3,Academia Dance Vision,3 Cl 7-89 Zona 8 Mixco,(+502) 5851 1116,NaN,NaN,ballet,Guatemala - Guatemala,"https://www.google.com.ar/maps/place/3+calle,+...",-90.576531,14.628877,"{'type': 'Point', 'coordinates': ['-90.5765311..."
4,Academia de Baile Español Mary Farrington,19 Calle 12-34,#ubicacion,NaN,Academia de Baile Español Mary Farrington te i...,Academias de Baile,Guatemala - Guatemala,https://www.google.com/maps/place/19+Calle+123...,-90.509229,14.589479,"{'type': 'Point', 'coordinates': ['-90.5092288..."


In [89]:
gymy['latlong'] = list(zip(gymy.lat, gymy.long))

In [77]:
test

[[-90.5072073, 14.5989133],
 2000,
 'Guatemala, Guatemala City',
 'oakland mall guatemala',
 '🇬🇹']

In [126]:
tuple(test[0])

(-90.5072073, 14.5989133)

In [167]:
test2 =(14.5989133,-90.5072073) 

In [168]:
#calculates distance between two points
def distancer(dfrow,user_input):
    return distance.distance(dfrow, user_input).km


In [172]:
gymy['distance'] = gymy.latlong.apply(lambda x , y=test2 : distance.distance(x, y).km)


In [122]:
gymy.head(1)

,names,address,phone,web,description,category,city,link,long,lat,principal,latlong,distance
0,180 Grados Fitness,Km. 22.5 Carretera a El Salvador,tel:+(502) 6662-5087,https://directorio.guatemala.com/listing_repor...,180 Grados Fitness tiene como objetivo fundame...,"Crossfit, TRX",Guatemala - Guatemala,https://www.google.com.ar/maps/place/180+grado...,-90.488614,14.50143,"{'type': 'Point', 'coordinates': ['-90.4886139...","(14.5014303, -90.4886139)",9.530515


In [145]:
#Function that gets the lat long from user input
def getting_coor_latlong():
    try:
        user_input = input('¿Donde te gustaría ejercitarte? ')
        kmeters = 2
        results = geocoder.geocode(user_input)
        fcoordinates=[]
        coordinates=[]
        coordinates.append(results[0]['geometry']['lat']) 
        coordinates.append(results[0]['geometry']['lng'])
        bandera = results[0]['annotations']['flag']
        country = results[0]['components']['country']
        city=results[0]['components']['city']
        localidad = country+', '+city
        fcoordinates.append(coordinates)
        fcoordinates.append(kmeters)
        fcoordinates.append(localidad)
        fcoordinates.append(user_input)
        fcoordinates.append(bandera)
        return fcoordinates
    except:
        return 'Not a valid address'

In [173]:
def main_geopy():
    try:
        coordenadas = getting_coor_latlong() #acá inicializa la funcion creada arriba
        latlong = coordenadas[0]
        radio = coordenadas[1]
        pais = coordenadas[4]
        localidad = coordenadas[2]
        direccion_solicitud = coordenadas[3]


        ratio= radio

        gymy_modelo = gymy.copy()
        #gymy_modelo['distance'] = gymy_modelo.latlong.apply(distancer)
        gymy_modelo['distance'] = gymy_modelo.latlong.apply(lambda x , y=latlong : distance.distance(x, y).km)
        display_df = gymy_modelo[gymy_modelo.distance<radio]
        display_df.reset_index(inplace=True)

        #creating map object
        tooltip = 'Location you chose: {} \n {}'.format(direccion_solicitud,localidad)
        mapa=folium.Map(latlong, zoom_start=14, width='100%', height='100%')
        folium.Marker(latlong, tooltip=tooltip, icon=folium.Icon()).add_to(mapa)
        for i in trange(len(display_df)):
            popup =''

            if str(display_df.phone[i]) != 'nan' :
                popup += 'Telefono: ' + str(display_df.phone[i])

            if str(display_df.address[i]) != 'nan' :
                popup += ' Direccion: '+str(display_df.address[i])

            if str(display_df.web[i]) != 'nan' :
                popup += ' Web: '+str(display_df.web[i])

            folium.Marker([display_df.lat[i],display_df.long[i]], popup=popup,
                          tooltip = display_df.names[i], icon = folium.Icon(color='red')).add_to(mapa)   
        mapa.save('MapaFinal.html')
        print('Amount of gyms near {} are: {}'.format(direccion_solicitud,n_gyms))
        display(mapa)
        display(display_df)

    except:
        print('Dirección inválida, por favor ingresa otra dirección')

In [174]:
main_geopy()

¿Donde te gustaría ejercitarte? zona 14 guatemala


100%|██████████| 30/30 [00:00<00:00, 1158.22it/s]


Amount of gyms near zona 14 guatemala are: 30


,index,names,address,phone,web,description,category,city,link,long,lat,principal,latlong,distance
0,1,Academia BellyDance Rashida,"Plaza Las Cañas, Local 201, zona 10, Ciudad de...",2337-0980,https://es-la.facebook.com/Rashidaacademy/,Esta academia de baile es ideal para las perso...,Baile,Guatemala - Guatemala,https://www.google.com.ar/maps/place/Academia+...,-90.505299,14.581305,"{'type': 'Point', 'coordinates': ['-90.5052988...","(14.5813049, -90.5052988)",1.434576
1,4,Academia de Baile Español Mary Farrington,19 Calle 12-34,#ubicacion,NaN,Academia de Baile Español Mary Farrington te i...,Academias de Baile,Guatemala - Guatemala,https://www.google.com/maps/place/19+Calle+123...,-90.509229,14.589479,"{'type': 'Point', 'coordinates': ['-90.5092288...","(14.5894785, -90.50922879999999)",1.481773
2,6,Academia Scenic Dance,3 Avenida 14-19 Zona 14,(+502) 5204 7081,http://www.scenicdance.com,NaN,ballet,Guatemala - Guatemala,https://www.google.com/maps/place/Scenic+Dance...,-90.529176,14.581688,"{'type': 'Point', 'coordinates': ['-90.5291758...","(14.581688, -90.52917579999999)",1.174405
3,23,Asha Shala,5a. Avenida 20-42 Zona 14,tel:+(502) 2367-2698,https://directorio.guatemala.com/listing_repor...,Aquí puedes encontrar la herramienta que neces...,Yoga,Guatemala - Guatemala,https://www.google.com/maps/place/Dharma+Heali...,-90.525663,14.572618,"{'type': 'Point', 'coordinates': ['-90.5256627...","(14.572618300000002, -90.5256627)",1.091099
4,26,Balance Concept Zona 13,Avenida Las Americas 19-30 Zona 13,(+502) 5809 1997,NaN,NaN,deportes,Guatemala - Guatemala,https://www.google.com/maps/place/Avenida+Las+...,-90.532289,14.577017,"{'type': 'Point', 'coordinates': ['-90.5322891...","(14.577016500000001, -90.5322891)",1.513424
5,29,Bilanz,18 Calle 20-70 Zona 10,tel:+(502) 2363-6392,NaN,Bilanz Pilates pretende que las personas que s...,Pilates TRX,Guatemala - Guatemala,https://www.google.com/maps/place/Bilanz+Pilat...,-90.516305,14.589540,"{'type': 'Point', 'coordinates': ['-90.5163053...","(14.58954, -90.5163053)",1.126722
6,31,Boom Gym,"15 Avenida 6-01, Century Plaza Nivel 4 Local 403",tel:+(502) 2362-0708,https://directorio.guatemala.com/listing_repor...,En Boom Gym nuestro personal está altamente ca...,Crossfit,Guatemala - Guatemala,https://www.google.com/maps/place/Centro+Comer...,-90.528258,14.584730,"{'type': 'Point', 'coordinates': ['-90.5282579...","(14.584730300000002, -90.5282579)",1.196433
7,33,Boom Kettlebell-Gym Carretera a El Salvador,Kilómetro 14.1 Carretera a El Salvador Centro,tel:+(502) 2339-5844,https://directorio.guatemala.com/listing_repor...,En Boom Kettlebell-Gym Carretera a El Salvador...,Crossfit,Guatemala - Guatemala,https://www.google.com/maps/place/Boom+Kettleb...,-90.508812,14.583239,"{'type': 'Point', 'coordinates': ['-90.5088124...","(14.5832395, -90.50881240000001)",1.118966
8,35,Boom Kettlebell-Gym Zona 10,"Boulevard Los Próceres, Plaza Las Cañas, Sótano 1",tel:+(502) 2337-3613,https://directorio.guatemala.com/listing_repor...,En Boom Kettlebell-Gym Zona 10 nuestro persona...,Crossfit,Guatemala - Guatemala,https://www.google.com/maps/place/Boom+Kettleb...,-90.508812,14.583239,"{'type': 'Point', 'coordinates': ['-90.5088124...","(14.5832395, -90.50881240000001)",1.118966
9,36,Brau Athletics CrossFit,"Plaza Empresarial Muxbal, Santa Catarina Pinula",2441 9008,https://www.facebook.com/crossfitspringcity7/,crossfit,crossfit,Guatemala - Guatemala,https://www.google.com/maps/place/Plaza+Empres...,-90.508812,14.583239,"{'type': 'Point', 'coordinates': ['-90.5088124...","(14.5832395, -90.50881240000001)",1.118966
